In [1]:
import sys
sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))

import os
os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')

import mlflow

from sem_covid.services.dataset_pipelines.unified_dataset_pipeline import UnifiedDatasetPipeline
from sem_covid.services.model_registry import embedding_registry
from sem_covid.services.store_registry import store_registry

In [2]:
EXPERIMENT_ID = '101'
BUCKET_NAME = 'mlflow'

In [3]:
# Embedding models
es_store = store_registry.es_index_store()
emb_model_use = embedding_registry.sent2vec_universal_sent_encoding()
emb_model_eurlex_bert = embedding_registry.doc2vec_eurlex_bert()

# Topic model
best_run = mlflow.search_runs(
    experiment_ids=EXPERIMENT_ID,
    order_by=['parameters.freq_topic_minus_1 ASC']
).iloc[0]

topic_model_bert = store_registry.minio_feature_store(BUCKET_NAME).get_features(
    features_name=f'{EXPERIMENT_ID}/{best_run.run_id}/artifacts/model/model.pkl')

INFO:absl:Using /tmp/tfhub_modules to cache modules.
Some weights of the model checkpoint at nlpaueb/bert-base-uncased-eurlex were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
unified_dataset_pipeline = UnifiedDatasetPipeline(es_store=es_store,
                                                  emb_model_1=emb_model_use,
                                                  emb_model_2=emb_model_eurlex_bert,
                                                  topic_model=topic_model_bert)
unified_dataset_pipeline.extract()
unified_dataset_pipeline.transform()
unified_dataset_pipeline.load()
